In [ ]:
from random import random

---
# Ejercicio 5
Considere una sucesión de números aleatorios $\{U_i\}_i$ y sea $M$ el primer $n$ tal que la variable $U_n$ es menor que su variable predecesora. Es decir,
$$
M = n \quad \text{tal que} \quad U_1 \leq U_2 \leq \ldots \leq U_{n-1} \quad \text{y } U_n < U_{n-1} 
$$

a) Justifique que $P(M > n) = \frac{1}{n!}, \quad n \geqslant 0$

Queremos hallar $P(M > n)$, probabilidad de que $M$, el primer $n$ tal que la variable $U_n$ es menor que su variable predecesora, sea *mayor estricto* que el índice de esa variable.

Tenemos que $\{U_i\}i$ es una sucesión de números aleatorios.

La sucesión va arrojando números aleatorios con distribución uniforme en el $(0,1)$, tal que se tienen:
$$
U_1, U_2, U_3, U_4, \ldots
$$

Supongamos por un momento que se almacenan en una lista tal que:
$$
[U_1, U_2, U_3, U_4, \ldots]
$$

Además supongamos que se generaron *n números aleatorios*:
$$
[U_1, U_2, U_3, U_4, \ldots, U_n]
$$

Ahora, podríamos a todos esos elementos permutarlos de *n formas distintas* con $n!$

Luego $P(M > n)$ significa que de esa lista de n permutaciones tenemos la combinación tal que:
$$
\text{Los } U_i \text{ están ordenados de menor a mayor} \wedge U_n < U_{n-1}
$$

Es decir 1 de las tantas combinaciones.

Luego esa combinación particular es 1 en $n!$

Luego $P(M > n ) = \frac{1}{n!}$

---
b) Utilice la identidad
$$
E[M] = \sum_{n=0}^\infty P(M > n)
$$

Para mostrar que $\displaystyle E[M] = e$

Tenemos por *Taylor* que:
$$
 e^x = \sum_{n=0}^\infty \frac{x^n}{n!}
$$


Supongamos que $x = 1 \Longrightarrow$
$$
 e = \sum_{n=0}^\infty \frac{1^n}{n!} = \sum_{n=0}^\infty \frac{1}{n!} \underset{\text{Ej (a)}}{=} \sum_{n=0}^\infty P(M>n) \underset{Identidad}{=} E[M]
$$

Luego $$\boxed{\displaystyle E[M] = e}$$

---
c) Utilice el resultado del item anterior para dar un estimador de $E[M]$, calcule el valor de su varianza
muestral. Mediante una simulación estime el valor de $e$ deteniéndose cuando la varianza muestral sea
menor que $0,01$.

In [ ]:
def M() -> int:
    
    n = 2
    #Caso Base
    U_prev = random()
    U = random()
    if U < U_prev: # En caso de que U_2 < U_1 devuelvo 2
        return n

    while U_prev < U: # General
        n += 1
        U_prev = U
        U = random()        
    return n